<a href="https://www.kaggle.com/code/nsff591/white-wine-data-engineering-rf-lightgbm-0-82?scriptVersionId=97815070" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Physical characteristics to determine wine quality

# Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np # making use of arrays
import matplotlib.pyplot as plt # plotting the data
import pandas as pd # importing the data

from sklearn.model_selection import train_test_split # splitting the data
from sklearn.preprocessing import StandardScaler, LabelEncoder # feature scaling

from sklearn.ensemble import RandomForestClassifier # Random forest model
import lightgbm as lgb # LightLGBM model
from sklearn.metrics import mean_squared_log_error # regression metric to analyse accuracy of the regression model
from sklearn.metrics import r2_score # another regression metric to analyse our accuracy
import seaborn as sns # to analyse the data before modelling
from sklearn.decomposition import KernelPCA # dimensional reduction technique
import pickle # this will allow us to save and load variables in a pickle file for later use
from tensorflow.keras.models import Sequential # deep learning library
from tensorflow.keras.layers import Dense, Dropout # deep learning library
from tensorflow import keras # optimizer in neural network
from sklearn.cluster import KMeans # kmeans clustering

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report # analysing the results
from sklearn.model_selection import cross_val_score # k-fold cross validation

print("Importing Complete!")

In [ ]:
pip install hyperopt

In [ ]:
from hyperopt import fmin, tpe, anneal, hp, space_eval, STATUS_OK #optimizing hyper parameters

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

# Importing the Dataset

In [ ]:
#importing the wine quality data
data = pd.read_csv('../input/white-wine-quality/winequality-white.csv', sep=';')
print(data.shape)

# deleting NaN rows
data = data.dropna()
#data = data.drop(['density','sulphates','pH'], axis=1)
print(str(data.shape) + " -> New shape after dropping NaN rows")

data.head(5)

# Analysing the Data

In [ ]:
feature_len = len(data.columns)-1

n = 0
rows = 3
columns = 4
i = 0
j = 0

fig, ax = plt.subplots(rows,columns,figsize= (10,6))

while i < rows:
  while j < columns:
    if n == feature_len:
      break 
    ax[i][j].hist(x = data.iloc[:,n].values,bins=20)
    ax[i][j].set_title(data.columns[n])
    n += 1
    j += 1
  i += 1
  j = 0

plt.tight_layout()
plt.show()

Analysing these histograms we can conclude several things:

* Alcohol value has a high variance as the data is spread out and barely aligning with a gaussian distribution
* Residual sugar is skewed right, I tried using a Log Transformation on it, but this did not improve the precision of models

In [ ]:
data.describe()

In [ ]:
feature_len = len(data.columns)-1

n = 0
rows = 3
columns = 4
i = 0
j = 0

fig, ax = plt.subplots(rows,columns,figsize= (10,6))

while i < rows:
  while j < columns:
    if n == feature_len:
      break 
    sns.barplot(x = 'quality', y = data.columns[n], data= data, ci= 95, ax= ax[i][j])
    ax[i][j].set_title(data.columns[n])
    n += 1
    j += 1
  i += 1
  j = 0

plt.tight_layout()
plt.show()


To further clarify the interpretation of these graphs: 


*   The black lines on the bars indicate that we can assure 95%(= confidence interval) of our data is within this number that our bar reaches.
*   If we see no change in the height of the bar(=mean) regardless of the quality, we can interpret little correlation between those features.
*   If we do see higher/lower height if the quality increases, we can assume higher correlation between these features.

We can conclude the following: 


1.   **Alcohol increases quality**
2.   **Chlorides decreases quality**
3.   **Density, Ph, Sulphates do not influence quality a lot**




In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(data.corr(), annot=True, robust=True)
plt.show()



1.   Density and Sugar 0.84 correlation (=high)
2.   Density and alcohol -0.78 reversed correlation (=high negative)
3.  **Alcohol has highest correlation of 0.44 with quality**


In [ ]:
def all_feature_scatter_plot():
  feature_len = len(data.columns)-1

  n = 0
  rows = 3
  columns = 4
  i = 0
  j = 0

  fig, ax = plt.subplots(rows,columns,figsize= (10,6))

  while i < rows:
    while j < columns:
      if n == feature_len:
        break 
      ax[i][j].scatter(y = data.iloc[:,n].values,x = data['quality'])
      ax[i][j].set_title(data.columns[n])
      n += 1
      j += 1
    i += 1
    j = 0

  plt.tight_layout()
  plt.show()

all_feature_scatter_plot()

As alcohol has the highest correlation with quality, we will heavily look at their outliers here.

*   **Alcohol has an outlier at the 9 quality mark**
*   free sulfur dioxide has an outlier at 3 quality (this might impact correlation with alcohol of -0.25)
*   Density, Fixed acidity, residual sugar and citric acid has some outliers at **6 quality**   

During my first tries on some models, I noticed a lower precision at quality 6. These outliers could explain it.



## Removing outliers

In [ ]:
import operator
#{'>': operator.gt,
#'<': operator.lt,
#'>=': operator.ge,
#'<=': operator.le,
#'==': operator.eq}

def remove_outlier(label, quality_number, target_cutoff, eq):
  n = 0
  lst = data.loc[:,[label,'quality']]
  lst_len = data.shape[0]

  while n < lst_len:
    if lst.iloc[n]['quality'] == quality_number and eq(lst.iloc[n][label],target_cutoff):
      print("old value: "+ str(data.loc[:,[label,'quality']].iloc[n][label]))
      data.loc[n,label] = data.groupby('quality').mean()[label][quality_number]
      print("new value: "+ str(data.loc[:,[label,'quality']].iloc[n][label]))
    n += 1

### Alcohol

In [ ]:
remove_outlier('alcohol', 9, 11, operator.le)

### Density

In [ ]:
remove_outlier('density', 6, 1.02, operator.ge)

### Residual sugar

In [ ]:
remove_outlier('residual sugar', 6, 40, operator.ge)

### Citric acid

In [ ]:
remove_outlier('citric acid', 6, 1.4, operator.ge)

### Fixed acidity

In [ ]:
remove_outlier('fixed acidity', 6, 12, operator.ge)

### Total sulfur dioxide

In [ ]:
remove_outlier('total sulfur dioxide', 3, 260, operator.ge)

### Free sulfur dioxide

In [ ]:
remove_outlier('free sulfur dioxide', 3, 190, operator.ge)

In [ ]:
all_feature_scatter_plot()

After removing most of the outliers we check the new scatterplot to confirm the imporoved data.

## Binning

Binning is a technique to split data in a feature into groups and replacing the original data with those groups.

Why would would you do that? Don't you lose information with it?

Yes, you lose information but you can make the model more robust, which can amplify the correlations in your model. The same principle is used in neural networks, where they drop a certain amount of data after a few hidden layers, to make the model more robust and less prune to overfitting.

### K-means algorithm to determine bins

I first thought about splitting the Alcohol data into 3 parts as it is easily distinguishable in the histogram. But after testing k-means clustering on it. I noticed 2 extra clusters which i wouldn't have before. (cluster 4 and 5 on the clustering graph below)

This led me to start coding the binning based on K-means clustering.

It turned out to be a good choice as it increased my accuracy by 10-15%. A HUGE INCREASE

Update: I later noticed I used Quality as part of my K-means algorithm. I shouldn't have done this as we are predicting the quality and we won't know it during the inferencing of our model. It could still be used to show the clusters and based on that we could hardcode some binning. But just remember that k-means shouldn't use the value we are predicting in the inferencing.

In [ ]:
alcohol = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 753)
    kmeans.fit(data.loc[:,['alcohol','quality']])
    alcohol.append(kmeans.inertia_)
plt.plot(range(1, 11), alcohol)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('alcohol')
plt.show()

I chose for 5 clusters as I noticed it to be a goog inflection point in the graph. I also tried 3 clusters, as this was my first precognition of the alcohol data but it returned 3-5% less accuracy so I reverted that idea.

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 951)
y_kmeans = kmeans.fit_predict(data.loc[:,['alcohol','quality']])

In [ ]:
k_means_data= data.loc[:,['alcohol','quality']]
plt.scatter(k_means_data.iloc[y_kmeans == 0, 0], k_means_data.iloc[y_kmeans == 0, 1], s = 100, c = 'red', label = 'Cluster 1')
plt.scatter(k_means_data.iloc[y_kmeans == 1, 0], k_means_data.iloc[y_kmeans == 1, 1], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(k_means_data.iloc[y_kmeans == 2, 0], k_means_data.iloc[y_kmeans == 2, 1], s = 100, c = 'green', label = 'Cluster 3')
plt.scatter(k_means_data.iloc[y_kmeans == 3, 0], k_means_data.iloc[y_kmeans == 3, 1], s = 100, c = 'cyan', label = 'Cluster 4')
plt.scatter(k_means_data.iloc[y_kmeans == 4, 0], k_means_data.iloc[y_kmeans == 4, 1], s = 100, c = 'magenta', label = 'Cluster 5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
plt.title('Cluster of Alcohol%')
plt.xlabel('Alcohol')
plt.ylabel('Quality')
plt.legend()
plt.show()

### Binning Alcohol

In [ ]:
def binning(label, n_clusters):
  kmeans = KMeans(n_clusters = n_clusters, init = 'k-means++', random_state = 951)
  y_kmeans = kmeans.fit_predict(data.loc[:,[label,'quality']])

  n = 0
  
  while n < data.shape[0]:
    data.loc[n,label] = y_kmeans[n]
    n += 1

binning('alcohol',5)

In [ ]:
plt.scatter(y = data['alcohol'],x = data['quality'])
plt.ylabel("Alcohol")
plt.xlabel("Quality")
plt.title("Alcohol after binning")
plt.show()

Notice how low alcohol still produced a high quality cluster. I would have never guessed this purely on a histogram.

In [ ]:
plt.hist(data['alcohol'])
plt.show()

**Binning alcohol had a big impact on accuracy**

Alcohol has the highest correlation with quality, which gave me the hint of binning it. This made the data more robust which increased Accuracy of the model in several models.

## Log Transformation

I wanted to use the log transformation on residual sugar and chloride but after some experiments, it did not improve the accuracy of the model. 

I did see 2 clusters of logged data appear in residual sugar and I could have binned this data, but I didn't think it was worth the time as I already showed that I know how to bin data.

# Splitting the data in training and test set + Shuffling

In [ ]:
# splitting the features(x) with the expected result(y) (quality of wine)
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=555)

print("Unscaled training data example:" + np.array2string(x_train[0], formatter={'float_kind':lambda x: "%.0f" % x}))

# Feauture Scaling

We feature scale as most models like K-means, calculate the distance between points and some features have vastly different ranges at which their numbers can appear. By scaling the features, it will diminish the difference and cause a better performance on the models.

We standardize instead of normalize because we want the outliers to have less of an effect. Standardisation lowers this effect. Normalisation amplifies it.

In [ ]:
# saving unscaled versions for testing purposes if needed
unscaled_x_train = x_train
unscaled_x_test = x_test

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

print("Scaled training data example:" + np.array2string(x_train[0], formatter={'float_kind':lambda x: "%.3f" % x}))

# Models: LightGBM model Classification

I wanted to try the lightGBM boosting model as it recently got a lot of popularity with it's fast performance and low memory usage. It is said that i will also perform better accuracy if you align your parameters correctly.

This is why I tried using it. 

It's weakness however, it is prune to overfitting.

I will use randomforrest model after this one because it is less prune to overfitting and has a low amount of parameters. On top of that, it uses bagging instead of boosting which means that I have 2 examples of ensemble methods.

## Optimizing LightGBM parameters using Hyperopt

Because LightGBM has an enormous amount of parameters, I used hyperopt to find the best one. But for those who don't want to go through the hurdle of optimizing hyper parameters. The default parameters gave me 1-2% less accuracy which is not that big of a difference.

In [ ]:
space = {'objective': 'multiclass',
         'metric': 'multi_logloss',
         'boosting': 'gbdt',
         'num_iterations': hp.choice('num_iterations', range(50, 2000, 5)),
         'learning_rate': hp.uniform('learning_rate', 0.003, 0.2),
         'num_leaves': hp.choice('num_leaves', range(5,150,1)),
         'max_depth': hp.choice('max_depth', range(2,10,1)),
         'min_data_in_leaf': hp.choice('min_data_in_leaf', range(10,400,5)),
         'reg_alpha': hp.uniform('reg_alpha',0,1),
         'reg_lambda': hp.uniform('reg_lambda',0,1),
         'n_estimators': hp.choice('n_estimators', range(50,2000,10))}

In [ ]:
accuracy_array= []

def objective(params):

  classifier_lgb = lgb.LGBMClassifier(**params)
  classifier_lgb.fit(x_train, y_train)
  y_pred=classifier_lgb.predict(x_test)
  accuracy = accuracy_score(y_test, y_pred)

  accuracy_array.append(accuracy)

  return {'loss': -accuracy, 'status': STATUS_OK }

best = fmin(objective,
    space=space,
    algo=anneal.suggest,
    max_evals=150)

# Print best parameters for anneal algo
best_params = space_eval(space, best)
print(best_params)

plt.plot(accuracy_array)
plt.show()

## Saving/loading best LightGBM parameters

In case you don't want to calculate the parameters everytime. I recommend loading the pickle file.

In [ ]:
pickle.dump(best_params, open( "./best_params_lightgbm.p", "wb" ))
pickle.dump(accuracy_array, open( "./accuracy_plot_lightgbm.p", "wb" ))

In [ ]:
best_params = pickle.load(open("./best_params_lightgbm.p", "rb" ))
accuracy_array = pickle.load(open("./accuracy_plot_lightgbm.p", "rb" ))

In [ ]:
plt.plot(accuracy_array)
plt.show()

## Model making/fitting

In [ ]:
classifier_lgb = lgb.LGBMClassifier(**best_params)

In [ ]:
classifier_lgb.fit(x_train, y_train)

In [ ]:
y_pred=classifier_lgb.predict(x_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
report=classification_report(y_test,y_pred)
print(report)

We can conclude that the precision is lacking at a wine quality of 4 and 6. **(This issue was partially resolved by binning alcohol)**

* 4 has a low sample size
* If we find another algorithm which can cover the unprecision of 6, we can make an ensemble model to alleviate this issue.

## Applying k-fold Cross validation

In [ ]:
accuracies = cross_val_score(estimator = classifier_lgb, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# Models: Random Forest Classifier

I stated in LightGBM, I chose randomforest because it is a bagging ensemble method and it is less prune to overfitting. On top of that, it is simple to use and almost always gives decent accuracy.

As a sidenote: I also tried XGBoost and SVM classifier but these perfermoded 5-8% worse even after hyperparameter optimization.

It is logical that SVM performed worse as it is "medium" sized data with relatively small number of features. Which SVM is bad at.

I did not optimize hyper parameters as it did not increase the accuracy by a significant amount

In [ ]:
classifier_rf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier_rf.fit(x_train, y_train)

In [ ]:
y_pred = classifier_rf.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
report=classification_report(y_test,y_pred)
print(report)

## Applying k-fold Cross validation

In [ ]:
accuracies = cross_val_score(estimator = classifier_rf, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

# Model: Neural network


I was thinking about using neural networks as they can tackle almost any problem. The big downside is it's computing time and unpredictable outcome. If the netowrk doesn't recognize the problem or structure, it might get stuck on the wrong foot. 

The network however works really well with missing data or outliers. But this is not the case for our data as we already removed most of those.

## Re-initializing data to make sure encoding is correctly applied for the neural network

We first need to change the quality feature to an encoded one where quality 5 = [0,0,1,0,0,0,0] for example. This is necessary for the neural network as the output layer will need 7 outputs in a this classification problem.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y= ohe.fit_transform(y.reshape(-1,1)).toarray()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=555)

print("Unscaled training data example:" + np.array2string(x_train[0], formatter={'float_kind':lambda x: "%.0f" % x}))

In [ ]:
# saving unscaled versions for testing purposes if needed
unscaled_x_train = x_train
unscaled_x_test = x_test

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

print("Scaled training data example:" + np.array2string(x_train[0], formatter={'float_kind':lambda x: "%.3f" % x}))

## Initializing the hidden layers

On why I chose certain parameters: I actuatly don't just tried a bit with trial and error. batch size is usually 32 and I noticed that i didn't need more than 50 epochs to converge on my model. So I made it 75 just to be sure.

The amount of hidden layers I added didn't change much to the accuracy, compared to having 2 before. I left them as it is, as it worked out somehow. (Yes I googled on how to improve it and I tried some, but it really didn't change a lot)

In [ ]:
batch_size = 32
val_split = 0.1
seed = 753

number_of_filters = 32
number_of_epochs = 75
number_of_units = 50

In [ ]:
nn = Sequential()

nn.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
nn.add(Dense(units=128,activation='relu'))
nn.add(Dense(units=64,activation='relu'))
nn.add(Dropout(0.25))
nn.add(Dense(units=64,activation='relu'))
nn.add(Dense(units=64,activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(units=32,activation='relu'))
nn.add(Dense(units=7,activation='softmax'))

I changed the learning rate to 0.0002 because otherwise the model would overfit way too quickly with the default adam value of 0.001.

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0002)
nn.compile(optimizer= opt, loss='categorical_crossentropy',metrics=['accuracy'])

## Fitting the model

In [ ]:
fitted_model= nn.fit(x_train,y_train, validation_split= val_split, epochs= number_of_epochs, batch_size= batch_size)

In [ ]:
y_pred = nn.predict(x_test)

pred = list()
for i in range(len(y_pred)):
  pred.append(np.argmax(y_pred[i]))

test = list()
for i in range(len(y_test)):
  test.append(np.argmax(y_test[i]))

In [ ]:
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
cm = confusion_matrix(test, pred)
print(cm)
print(accuracy_score(test, pred))

In [ ]:
report=classification_report(test,pred)
print(report)

## Analysing the model on overfitting

In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    try:
      name = metric.replace("_"," ").capitalize()
      plt.plot(history.epoch, history.history[metric], label='Train')
      plt.plot(history.epoch, history.history['val_'+metric], linestyle="--", label='Val')
      plt.xlabel('Epoch')
      plt.ylabel(name)
      if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
      elif metric == 'auc':
        plt.ylim([0.8,1])
      else:
        plt.ylim([0,1])
      plt.legend()
      plt.show()  
    except:
      pass
plot_metrics(fitted_model)

plt.title(label='Zoomed in Accuracy plot')
plt.plot(fitted_model.history["accuracy"],label='Train')
plt.plot(fitted_model.history["val_accuracy"],linestyle="--",label='Validation')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

Our training accuracy is very close to our validation accuracy. This is a good indicator that we did not overfit our model.

# Conclusion


## Model Comparison

Model performance compared with **K-fold Cross Validation**:

1.   Random forest -> Accuracy: **81.98 %** Standard Deviation: 1.50 %
2.   lightGBM -> Accuracy: **81.42 %** Standard Deviation: 1.35 %
3.   Neural network -> Accuracy is: **77.55 %**

Although Random forest won with accuracy, lightGBM has a lower standard deviation. So I would compare them to be roughly the same accuracy as in some applications a lower Standard deviation is desired.

The neural network is the big loser here. Why?

If i let the neural network go for 200-500 epochs, I could get a higher accuracy but this would overfit the model drasticly.

How could I still improve the neural network without overfitting? My suggestion would be to increase the dataset. Neural networks work well with lots of data. 5000 rows where most of the features are very uncorrolated to the quality is just too little for the network to work efficiently on. 

I tried deleting uncorrolated features but this only lowered the accuracy about 1-2%, which is the undesired effect.

I think I could still increase the accuracy of the Randomforest and LightGBM models if I data engineered more. An idea of mine was to log transform the residual sugar and then bin it into multiple groups. This might also work for density as it has a high corelation with alcohol.

## Comparing the parameters

I tried a lot of parameters manually on LightGBM but most of them resulted in the same outcome. Only after using hyperopt to optimise the parameters did I increase it's accuracy by 2%.

RandomForest didn't need much parameter tuning, as the default settings almost always were the best choice.

I do think that I could tune the neural network parameters more as I was pretty lousy on that front and didn't try using hyperopt.

## Noteworthy experiences

##### When I first put the raw data into the models, I received accuracies between 55% and 65%, which was horrible. After optimizing the parameters, it increased to 69-72%. And once I got to data engineering, mainly the binning of alcohol, it increased to 80-82%.

As many start their AI journey focussing on model improvement, they often forget the importance of data engineering. This is why I would recommend people to more analyse the Data and see if you can change it to your adventage.